In [35]:
import sys  
sys.path.insert(1, './../../../ESN-IP/IP/.')

import random

random.seed(10)

from DATA import NARMA10

data = NARMA10(True, [80,0,20])

X_TR, Y_TR = data.TR()
X_VAL, Y_VAL = data.VAL()
X_TS, Y_TS = data.TS()


In [36]:
from Metrics import MSE
from IPReservoir import IPReservoir
from ESN import Reservoir, EchoStateNetwork
from IntrinsicPlasticity import IPMask, IPDistribution

N_UNITS_LIST = [100, 150, 200, 250]
W_range=(-0.5, 0.5)
initial_rho = 0.5
desired_rhos = [0.95, 0.95, 0.95, 0.95]
bias_range = [-0.1, 0.1]


In [37]:
REPETITIONS = 2
max_epochs = 15 
columns = ["Name", "TYPE", "MSE", "MC", "Lambda max", "delta_phI", "KL", "Rho", "A-Norm", "B-Norm"]
models_num = len(N_UNITS_LIST)*len(desired_rhos)
rows = []

for N_UNITS, desired_rho in zip(N_UNITS_LIST, desired_rhos):
    config_string = f"{N_UNITS} UNIT - Spectral Radius: {desired_rho}"
    names = ["Vanilla", "Gauss 1", "Gauss 2", "Bimodal", "Trim 1", "Trim 2"]
    identifiers = [ f"- {name} - {config_string} " for name in names]
    
    for R in range(REPETITIONS):
        vanilla_reservoir = Reservoir(N = N_UNITS, ro_rescale = desired_rho, W_range=W_range, bias_range=bias_range)
        
        gaussian_reservoir_1 = IPReservoir(N = N_UNITS, ro_rescale=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.gaussian(N_UNITS, 0.5, apply_activation=True))
        gaussian_reservoir_2 = IPReservoir(N = N_UNITS, ro_rescale=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.gaussian(N_UNITS, 0.25, apply_activation=True))

        bimodal_reservoir = IPReservoir(N = N_UNITS, ro_rescale=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.mixedBimodalMask(N_UNITS, 0.69, 0.92, apply_activation=True))
        
        trimodal_reservoir_1 = IPReservoir(N = N_UNITS, ro_rescale=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.trimodal(N_UNITS, 9/10, 0.25, 0.15, 0.92, apply_activation=True))
        trimodal_reservoir_2 = IPReservoir(N = N_UNITS, ro_rescale=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.trimodal(N_UNITS, 9/10, 0.25, 0.075, 0.72, apply_activation=False))
        
        print("Traning IP models")
        gaussian_reservoir_1.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, verbose=False)
        gaussian_reservoir_2.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, verbose=False)
        bimodal_reservoir.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, verbose=False)
        trimodal_reservoir_1.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=0.000005, verbose=False)
        trimodal_reservoir_2.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=0.000001, verbose=False)

        models = [vanilla_reservoir, gaussian_reservoir_1, gaussian_reservoir_2, bimodal_reservoir, trimodal_reservoir_1, trimodal_reservoir_2]

        for model_index in range(len(models)): 
            model = models[model_index]

            esn = EchoStateNetwork(model)
            esn.train(X_TR,Y_TR, 0)

            mse = esn.evaluate(X_TS, Y_TS, metric=MSE()).item()
            
            mc = esn.MemoryCapacity().item()
            
            lambda_max = model.LCE(X_TS)

            de_phi = model.de_fi()

            KL = model.kl_value.item() if model_index != 0 else None
            a_norm = model.a.norm().item() if model_index != 0 else 1
            b_norm = model.b.norm().item() if model_index != 0 else 0

            # columns = ["________________Name____________________","TYPE", "MSE","MC","Lambda max","delta_phI","KL", "Rho", "A-Norm", "B-Norm"]
            rows.append([f"{R+1}) {identifiers[model_index]}" , model_index, mse, mc, lambda_max, de_phi, KL, model.max_eigs().item(), a_norm, b_norm])
            print(rows[-1])

        
        

Traning IP models
Reservoir warmed up with the first 100 time steps
['1) - Vanilla - 100 UNIT - Spectral Radius: 0.95 ', 0, 0.0216264421662073, 16.712900161743164, -0.07732938, 0.06899994371685869, None, 0.9499998688697815, 1, 0]
Reservoir warmed up with the first 100 time steps


C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.19041e-11): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.23113e-11): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.14437e-11): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Lo

['1) - Gauss 1 - 100 UNIT - Spectral Radius: 0.95 ', 1, 0.021479941858597758, 9.424439430236816, -0.054575693, 0.056938006758242656, 0.24935327470302582, 0.94969242811203, 16.899097442626953, 4.760641098022461]
Reservoir warmed up with the first 100 time steps


C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.29797e-11): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=7.63913e-12): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=5.01993e-11): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Lo

['1) - Gauss 2 - 100 UNIT - Spectral Radius: 0.95 ', 2, 0.02161291973939263, 8.865239143371582, -0.14141451, 0.04788066842138881, 0.20034655928611755, 0.9437028765678406, 16.2039737701416, 6.010500431060791]
Reservoir warmed up with the first 100 time steps


...

['1) - Vanilla - 250 UNIT - Spectral Radius: 0.95 ', 0, 0.02203137468481006, 22.596521377563477, -0.15891844, 0.05471547431164143, None, 0.9499983191490173, 1, 0]
Reservoir warmed up with the first 100 time steps
['1) - Gauss 1 - 250 UNIT - Spectral Radius: 0.95 ', 1, 0.02288565115439263, 11.34427547454834, -0.16083264, 0.06792397267672928, 0.24224944412708282, 0.9473841786384583, 27.21631622314453, 7.70412540435791]
Reservoir warmed up with the first 100 time steps
['1) - Gauss 2 - 250 UNIT - Spectral Radius: 0.95 ', 2, 0.10814963753079887, 10.124284744262695, -0.37768438, 0.14403735835667986, 0.23909084498882294, 0.947933554649353, 28.15757942199707, 11.297135353088379]
Reservoir warmed up with the first 100 time steps


C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.19769e-12): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.45871e-12): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.78033e-12): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Lo

['1) - Bimodal - 250 UNIT - Spectral Radius: 0.95 ', 3, 0.022184554918611384, 9.23514175415039, -0.33255363, 0.06382397558033744, 0.10097671300172806, 0.9491986632347107, 30.011743545532227, 14.971890449523926]
Reservoir warmed up with the first 100 time steps
['1) - Trim 1 - 250 UNIT - Spectral Radius: 0.95 ', 4, 0.02202501441939263, 11.029372215270996, -0.06181847, 0.0755169858476971, 0.09405829012393951, 0.9493106603622437, 28.902896881103516, 13.515384674072266]
Reservoir warmed up with the first 100 time steps
['1) - Trim 2 - 250 UNIT - Spectral Radius: 0.95 ', 5, 0.022275278945486376, 11.426251411437988, -0.06079576, 0.11669559440579683, 0.0551433190703392, 0.9451707601547241, 31.422317504882812, 13.775566101074219]
Traning IP models
Reservoir warmed up with the first 100 time steps


C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.17179e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


['2) - Vanilla - 250 UNIT - Spectral Radius: 0.95 ', 0, 0.021983363481337708, 23.03170394897461, -0.0617056, 0.0486662878522137, None, 0.949999988079071, 1, 0]
Reservoir warmed up with the first 100 time steps
['2) - Gauss 1 - 250 UNIT - Spectral Radius: 0.95 ', 1, 0.022144171861111377, 10.933096885681152, -2.539224, 0.46333897496888254, 0.25625354051589966, 0.9467499852180481, 27.2808780670166, 7.585505485534668]
Reservoir warmed up with the first 100 time steps
['2) - Gauss 2 - 250 UNIT - Spectral Radius: 0.95 ', 2, 0.023479697804236376, 10.467204093933105, -0.3656667, 0.08169640670630977, 0.21068738400936127, 0.8970606327056885, 26.746753692626953, 10.120939254760742]
Reservoir warmed up with the first 100 time steps
['2) - Bimodal - 250 UNIT - Spectral Radius: 0.95 ', 3, 0.02285206593918267, 9.75821590423584, -0.5246183, 0.2687752469867959, 0.1004650667309761, 0.9456153512001038, 29.505380630493164, 14.315189361572266]
Reservoir warmed up with the first 100 time steps
['2) - Trim 1

In [39]:
import pandas as pd 
df = pd.DataFrame(rows)
df.to_csv("./RESULTS_NARMA10.csv", header=columns)

In [42]:
df = pd.DataFrame(rows, columns=columns)

In [43]:
df.groupby("TYPE").mean()

,MSE,MC,Lambda max,delta_phI,KL,Rho,A-Norm,B-Norm
TYPE,,,,,,,,
0,0.021832,20.968672,-0.119658,0.063288,NaN,0.950000,1.000000,0.000000
1,0.022020,10.310652,-0.920700,0.241393,0.251774,0.943127,22.615183,6.309521
2,0.033346,9.549275,-1.379600,0.127523,0.227529,0.937140,23.068914,8.904370
3,0.021907,9.002080,-0.791319,0.188575,0.097414,0.947827,24.938432,12.554076
4,0.021839,10.401893,-0.273329,0.127452,0.083611,0.944889,24.570249,10.895304
5,0.021876,10.737753,-0.091002,0.087818,0.056141,0.945963,25.500667,10.932029
